In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load the dataset
url = "C:\\Users\\DELL\\sentiment analysis\\synthetic_healthcare_reviews_with_text.csv"   #url = r"C:\Users\DELL\sentiment analysis\synthetic_healthcare_reviews_with_text.csv"      #url = "C:/Users/DELL/sentiment analysis/synthetic_healthcare_reviews_with_text.csv"
df = pd.read_csv(url)

In [ ]:
# Remove rows where 'Review_Text' is null
data_cleaned = df.dropna(subset=['Review_Text'])

# Display the cleaned DataFrame
print(data_cleaned)

# Save the cleaned data to a new CSV file
data_cleaned.to_csv('cleaned_data_sentiment_analysis.csv', index=False)

In [5]:
# Load the dataset
url_1 = "C:\\Users\\DELL\\sentiment analysis\\cleaned_data_sentiment_analysis.csv"   #url = r"C:\Users\DELL\sentiment analysis\synthetic_healthcare_reviews_with_text.csv"      #url = "C:/Users/DELL/sentiment analysis/synthetic_healthcare_reviews_with_text.csv"
df = pd.read_csv(url_1)

In [ ]:
def clean_text(text):
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()
    return text

# Apply the cleaning function to the review column, if it exists
if 'Review_Text' in df.columns:
    df['cleaned_review'] = df['Review_Text'].apply(clean_text)
    print(df[['Review_Text', 'cleaned_review']].head())
else:
    print("The column 'review' does not exist in the dataset.")

In [ ]:
print("Columns in the dataset:", df.columns)

In [ ]:
def map_rating_to_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

# Apply the function to the 'Rating' column to create a new 'Sentiment' column
df['Sentiment'] = df['Rating'].apply(map_rating_to_sentiment)

# Optionally, you can drop the original 'Rating' column if it's no longer needed
# data.drop('Rating', axis=1, inplace=True)

# Display the dataframe with the new 'Sentiment' column
print(df[['cleaned_review','Rating', 'Sentiment']])

In [ ]:
# Dropping the 'Review_Text' column
df = df.drop(columns=['Rating'])

# Display the resulting DataFrame
print(df)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_data_sentiment_analysis_fine.csv', index=False)

In [ ]:
import pandas as pd
from textblob import TextBlob

# Assuming your DataFrame is already loaded as 'df'
# df = pd.read_csv('cleaned_data_sentiment_analysis_fine.csv')

# Function to calculate polarity
def get_polarity(text):
    return TextBlob(text).sentiment.polarity

# Apply the function to the 'cleaned_review' column and create a new 'polarity' column
df['polarity'] = df['cleaned_review'].apply(get_polarity)

# Display the first few rows of the DataFrame with polarity
print(df.head())
# Save the cleaned data to a new CSV file
df.to_csv('cleaned_data_sentiment_analysis_fine_polarity.csv', index=False)

In [ ]:
#Preparing Data
url_2 = "C:\\Users\\DELL\\sentiment analysis\\cleaned_data_sentiment_analysis_fine_polarity.csv" 
df = pd.read_csv(url_2)
import pandas as pd
# Assume 'review_text' contains the text reviews and 'rating' contains the numerical ratings
# Define conditions
positive_condition = (df['polarity'] > 0) & (df['Sentiment'] == 'Positive')
negative_condition = (df['polarity'] < 0) & (df['Sentiment'] == 'Negative')
neutral_condition = (df['polarity'] >= -0.2) & (df['polarity'] <= 0.2) & (df['Sentiment'] == 'Neutral')

# Filter the data based on the conditions
filtered_df = df[positive_condition | negative_condition | neutral_condition]

# Display the filtered DataFrame
print(filtered_df)
print(filtered_df['Sentiment'].value_counts())

# Save the cleaned data to a new CSV file
filtered_df.to_csv('cleaned_data_sentiment_analysis_fine_polarity_cleaned.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Convert text data into features (assuming you have preprocessed text data in 'cleaned_review' column)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=2, max_df=0.8, stop_words='english')
X_tfidf = tfidf.fit_transform(filtered_df['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_df['Sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize the classifier
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:

print(filtered_df['Sentiment'].value_counts())

In [ ]:
# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import precision_score

# Calculate precision score with different averaging methods
precision = precision_score(y_test, y_pred, average='macro')
print(f'Macro-Averaged Precision: {precision:.2f}')

precision_micro = precision_score(y_test, y_pred, average='micro')
print(f'Micro-Averaged Precision: {precision_micro:.2f}')

precision_weighted = precision_score(y_test, y_pred, average='weighted')
print(f'Weighted-Averaged Precision: {precision_weighted:.2f}')


In [ ]:
from sklearn.metrics import recall_score

# Calculate recall score with different averaging methods
recall_macro = recall_score(y_test, y_pred, average='macro')
print(f'Macro-Averaged Recall: {recall_macro:.2f}')

recall_micro = recall_score(y_test, y_pred, average='micro')
print(f'Micro-Averaged Recall: {recall_micro:.2f}')

recall_weighted = recall_score(y_test, y_pred, average='weighted')
print(f'Weighted-Averaged Recall: {recall_weighted:.2f}')


In [ ]:
y = label_encoder.transform(filtered_df['Sentiment'])

# Plot class distribution
plt.figure(figsize=(8, 5))
pd.Series(y).value_counts().plot(kind='bar', color='coral')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Number of Instances')
plt.xticks(ticks=[0, 1, 2], labels=label_encoder.classes_)
plt.show()

In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score


# Separate the data into different classes
df_positive = filtered_df[filtered_df['Sentiment'] == 'Positive']
df_neutral = filtered_df[filtered_df['Sentiment'] == 'Neutral']
df_negative = filtered_df[filtered_df['Sentiment'] == 'Negative']

# Perform oversampling on the neutral class

df_neutral_oversampled = resample(df_neutral, 
                                  replace=True,    # Sample with replacement
                                  n_samples=len(df_negative),  # Match number of samples in positive class
                                  random_state=42)  # For reproducibility
df_positive_oversampled = resample(df_positive, 
                                  replace=True,    # Sample with replacement
                                  n_samples=len(df_negative),  # Match number of samples in positive class
                                  random_state=42)  # For reproducibility


# Combine the oversampled neutral data with the original positive and negative data
df_oversampled = pd.concat([df_positive_oversampled, df_neutral_oversampled, df_negative])

# Shuffle the dataset to mix the samples
df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the class distribution after oversampling
print(df_oversampled)

# Convert text data into features (assuming you have preprocessed text data in 'cleaned_review' column)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=2, max_df=0.8, stop_words='english')
X_tfidf = tfidf.fit_transform(df_oversampled['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_oversampled['Sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize the classifier
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


new_reviews = ["The doctor was incredibly attentive and took the time to explain everything.", "I had a great experience at the hospital; the staff were kind and professional.", "The surgery went smoothly, and the recovery process was well-managed.", "The nurses were compassionate and made me feel comfortable during my stay.", "I was seen quickly, and the treatment was effective. Highly recommend this clinic.", "The telemedicine service was very convenient, and the doctor was knowledgeable.", "I felt well cared for throughout my entire hospital stay. Excellent service!", "The facility was clean, and the staff were always ready to help.", "My appointment was on time, and the staff made sure I was comfortable.", "The healthcare team was professional, and I received top-notch care.", "The appointment was okay, but the wait time was a bit long.", "The service was decent, but I’ve had better experiences at other clinics.", "The treatment was adequate, but I felt it could have been more personalized.", "The staff were polite, but the environment felt a bit rushed.", "The care was satisfactory, but nothing stood out as exceptional.", "The hospital visit was fine, but the paperwork process was slow.", "The diagnosis was accurate, but the explanation could have been clearer.", "The facilities were clean, but the overall atmosphere was somewhat sterile.", "I received the treatment I needed, but the experience was just average.", "The consultation was helpful, but the follow-up could have been better.", "The wait time was unbearable, and the staff seemed disorganized.", "I was unhappy with the treatment plan; it felt rushed and impersonal.", "The hospital was understaffed, and I didn’t feel like I received adequate care.", "The nurse was rude and unprofessional during my visit.", "I had a terrible experience; the doctor didn’t listen to my concerns.", "The clinic was dirty, and the equipment looked outdated.", "I was misdiagnosed, which led to further complications.", "The receptionist was unhelpful and made the check-in process stressful.", "I felt neglected during my hospital stay; the staff were not attentive.", "The online consultation was frustrating, and I didn’t get the answers I needed.", "The doctor was knowledgeable, but the appointment felt rushed.", "The hospital staff were friendly, but the billing process was confusing.", "I appreciated the doctor’s expertise, but the wait time was too long.", "The care was good, but the hospital food was terrible.", "The staff were professional, but I didn’t feel a personal connection.", "The treatment worked, but the side effects weren’t explained to me.", "The nurse was helpful, but the room was not clean.", "The doctor was excellent, but the clinic’s location was inconvenient.", "The service was prompt, but I felt like just another number.", "The care was adequate, but the follow-up was lacking.", "The specialist was excellent, but getting an appointment was difficult.", "The telemedicine service was useful, but the connection quality was poor.", "The doctor was kind, but the overall experience was just okay.", "The hospital was modern, but the staff seemed overworked.", "The surgery was successful, but the aftercare instructions were confusing.", "The clinic was well-equipped, but the staff weren’t very friendly.", "The treatment was effective, but I didn’t feel well-informed about my options.", "The hospital stay was fine, but the noise level was too high.", "The doctor was caring, but the administrative staff were unhelpful.", "The healthcare service was good, but the waiting area was uncomfortable."
]

# Ensure the vectorizer is not refitted, only transform the new data
new_tfidf_features = tfidf.transform(new_reviews)

# Predict sentiment using the trained RandomForestClassifier model
new_predictions = model.predict(new_tfidf_features)

# Decode the numeric predictions back to original labels
decoded_predictions = label_encoder.inverse_transform(new_predictions)

print(decoded_predictions)


In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import label_binarize


#
print(filtered_df['Sentiment'].value_counts())


# Initialize the SVM classifier with probability=True
model = SVC(probability=True, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')
print(f'Precision: {precision * 100:.2f}%')

# Calculate Recall
recall = recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall * 100:.2f}%')

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

# Calculate ROC-AUC
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])  # Assuming 3 classes: 0=Negative, 1=Neutral, 2=Positive
y_pred_proba = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')
print(f'ROC-AUC: {roc_auc * 100:.2f}%')


new_reviews = ["The doctor was incredibly attentive and took the time to explain everything.", "I had a great experience at the hospital; the staff were kind and professional.", "The surgery went smoothly, and the recovery process was well-managed.", "The nurses were compassionate and made me feel comfortable during my stay.", "I was seen quickly, and the treatment was effective. Highly recommend this clinic.", "The telemedicine service was very convenient, and the doctor was knowledgeable.", "I felt well cared for throughout my entire hospital stay. Excellent service!", "The facility was clean, and the staff were always ready to help.", "My appointment was on time, and the staff made sure I was comfortable.", "The healthcare team was professional, and I received top-notch care.", "The appointment was okay, but the wait time was a bit long.", "The service was decent, but I’ve had better experiences at other clinics.", "The treatment was adequate, but I felt it could have been more personalized.", "The staff were polite, but the environment felt a bit rushed.", "The care was satisfactory, but nothing stood out as exceptional.", "The hospital visit was fine, but the paperwork process was slow.", "The diagnosis was accurate, but the explanation could have been clearer.", "The facilities were clean, but the overall atmosphere was somewhat sterile.", "I received the treatment I needed, but the experience was just average.", "The consultation was helpful, but the follow-up could have been better.", "The wait time was unbearable, and the staff seemed disorganized.", "I was unhappy with the treatment plan; it felt rushed and impersonal.", "The hospital was understaffed, and I didn’t feel like I received adequate care.", "The nurse was rude and unprofessional during my visit.", "I had a terrible experience; the doctor didn’t listen to my concerns.", "The clinic was dirty, and the equipment looked outdated.", "I was misdiagnosed, which led to further complications.", "The receptionist was unhelpful and made the check-in process stressful.", "I felt neglected during my hospital stay; the staff were not attentive.", "The online consultation was frustrating, and I didn’t get the answers I needed.", "The doctor was knowledgeable, but the appointment felt rushed.", "The hospital staff were friendly, but the billing process was confusing.", "I appreciated the doctor’s expertise, but the wait time was too long.", "The care was good, but the hospital food was terrible.", "The staff were professional, but I didn’t feel a personal connection.", "The treatment worked, but the side effects weren’t explained to me.", "The nurse was helpful, but the room was not clean.", "The doctor was excellent, but the clinic’s location was inconvenient.", "The service was prompt, but I felt like just another number.", "The care was adequate, but the follow-up was lacking.", "The specialist was excellent, but getting an appointment was difficult.", "The telemedicine service was useful, but the connection quality was poor.", "The doctor was kind, but the overall experience was just okay.", "The hospital was modern, but the staff seemed overworked.", "The surgery was successful, but the aftercare instructions were confusing.", "The clinic was well-equipped, but the staff weren’t very friendly.", "The treatment was effective, but I didn’t feel well-informed about my options.", "The hospital stay was fine, but the noise level was too high.", "The doctor was caring, but the administrative staff were unhelpful.", "The healthcare service was good, but the waiting area was uncomfortable."
]

# Ensure the vectorizer is not refitted, only transform the new data
new_tfidf_features = tfidf.transform(new_reviews)

# Predict sentiment using the trained model (best_model)
new_predictions = best_model.predict(new_tfidf_features)

# Decode the numeric predictions back to original labels
decoded_predictions = label_encoder.inverse_transform(new_predictions)

print(decoded_predictions)


In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import label_binarize




# Convert text data into features (assuming you have preprocessed text data in 'cleaned_review' column)
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=2, max_df=0.8, stop_words='english')
X_tfidf = tfidf.fit_transform(filtered_df['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_df['Sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]  # Used only for 'poly' kernel
}

# Initialize GridSearchCV with SVM
grid_search = GridSearchCV(SVC(probability=True, random_state=42), param_grid, cv=3, n_jobs=-1, verbose=2)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy after hyperparameter tuning: {accuracy * 100:.2f}%')

# Calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')
print(f'Precision: {precision * 100:.2f}%')

# Calculate Recall
recall = recall_score(y_test, y_pred, average='weighted')
print(f'Recall: {recall * 100:.2f}%')

# Calculate F1 Score
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {f1 * 100:.2f}%')

# Calculate ROC-AUC
y_test_binarized = label_binarize(y_test, classes=[0, 1, 2])  # Assuming 3 classes: 0=Negative, 1=Neutral, 2=Positive

# Use best_model for prediction probabilities
y_pred_proba = best_model.predict_proba(X_test)

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, average='weighted', multi_class='ovr')
print(f'ROC-AUC: {roc_auc * 100:.2f}%')


new_reviews = ["The doctor was incredibly attentive and took the time to explain everything.", "I had a great experience at the hospital; the staff were kind and professional.", "The surgery went smoothly, and the recovery process was well-managed.", "The nurses were compassionate and made me feel comfortable during my stay.", "I was seen quickly, and the treatment was effective. Highly recommend this clinic.", "The telemedicine service was very convenient, and the doctor was knowledgeable.", "I felt well cared for throughout my entire hospital stay. Excellent service!", "The facility was clean, and the staff were always ready to help.", "My appointment was on time, and the staff made sure I was comfortable.", "The healthcare team was professional, and I received top-notch care.", "The appointment was okay, but the wait time was a bit long.", "The service was decent, but I’ve had better experiences at other clinics.", "The treatment was adequate, but I felt it could have been more personalized.", "The staff were polite, but the environment felt a bit rushed.", "The care was satisfactory, but nothing stood out as exceptional.", "The hospital visit was fine, but the paperwork process was slow.", "The diagnosis was accurate, but the explanation could have been clearer.", "The facilities were clean, but the overall atmosphere was somewhat sterile.", "I received the treatment I needed, but the experience was just average.", "The consultation was helpful, but the follow-up could have been better.", "The wait time was unbearable, and the staff seemed disorganized.", "I was unhappy with the treatment plan; it felt rushed and impersonal.", "The hospital was understaffed, and I didn’t feel like I received adequate care.", "The nurse was rude and unprofessional during my visit.", "I had a terrible experience; the doctor didn’t listen to my concerns.", "The clinic was dirty, and the equipment looked outdated.", "I was misdiagnosed, which led to further complications.", "The receptionist was unhelpful and made the check-in process stressful.", "I felt neglected during my hospital stay; the staff were not attentive.", "The online consultation was frustrating, and I didn’t get the answers I needed.", "The doctor was knowledgeable, but the appointment felt rushed.", "The hospital staff were friendly, but the billing process was confusing.", "I appreciated the doctor’s expertise, but the wait time was too long.", "The care was good, but the hospital food was terrible.", "The staff were professional, but I didn’t feel a personal connection.", "The treatment worked, but the side effects weren’t explained to me.", "The nurse was helpful, but the room was not clean.", "The doctor was excellent, but the clinic’s location was inconvenient.", "The service was prompt, but I felt like just another number.", "The care was adequate, but the follow-up was lacking.", "The specialist was excellent, but getting an appointment was difficult.", "The telemedicine service was useful, but the connection quality was poor.", "The doctor was kind, but the overall experience was just okay.", "The hospital was modern, but the staff seemed overworked.", "The surgery was successful, but the aftercare instructions were confusing.", "The clinic was well-equipped, but the staff weren’t very friendly.", "The treatment was effective, but I didn’t feel well-informed about my options.", "The hospital stay was fine, but the noise level was too high.", "The doctor was caring, but the administrative staff were unhelpful.", "The healthcare service was good, but the waiting area was uncomfortable."
]

# Ensure the vectorizer is not refitted, only transform the new data
new_tfidf_features = tfidf.transform(new_reviews)

# Predict sentiment using the trained model (best_model)
new_predictions = best_model.predict(new_tfidf_features)

# Decode the numeric predictions back to original labels
decoded_predictions = label_encoder.inverse_transform(new_predictions)

print(decoded_predictions)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Count the occurrences of each predicted sentiment
sentiment_counts = np.unique(decoded_predictions, return_counts=True)

# Create a bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x=sentiment_counts[0], y=sentiment_counts[1], palette='viridis')
plt.title('Distribution of Predicted Sentiments')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.show()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts[1], labels=sentiment_counts[0], autopct='%1.1f%%', startangle=140, colors=sns.color_palette('viridis'))
plt.title('Predicted Sentiment Distribution')
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

# Convert text data into features (assuming you have preprocessed text data in 'cleaned_review' column)
from sklearn.feature_extraction.text import TfidfVectorizer


# Separate the data into different classes
df_positive = filtered_df[filtered_df['Sentiment'] == 'Positive']
df_neutral = filtered_df[filtered_df['Sentiment'] == 'Neutral']
df_negative = filtered_df[filtered_df['Sentiment'] == 'Negative']

# Perform oversampling on the neutral class

df_neutral_oversampled = resample(df_neutral, 
                                  replace=True,    # Sample with replacement
                                  n_samples=len(df_negative),  # Match number of samples in positive class
                                  random_state=42)  # For reproducibility
df_positive_oversampled = resample(df_positive, 
                                  replace=True,    # Sample with replacement
                                  n_samples=len(df_negative),  # Match number of samples in positive class
                                  random_state=42)  # For reproducibility


# Combine the oversampled neutral data with the original positive and negative data
df_oversampled = pd.concat([df_positive_oversampled, df_neutral_oversampled, df_negative])

# Shuffle the dataset to mix the samples
df_oversampled = df_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the class distribution after oversampling
print(df_oversampled)

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=1, max_df=0.9, stop_words='english')
X_tfidf = tfidf.fit_transform(df_oversampled['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_oversampled['Sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)


# Initialize the Logistic Regression classifier
logreg_model = LogisticRegression(max_iter=1000)

# Train the model
logreg_model.fit(X_train, y_train)

# Make predictions
logreg_predictions = logreg_model.predict(X_test)

# Calculate accuracy
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
print(f'Logistic Regression Accuracy: {logreg_accuracy * 100:.2f}%')



In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# TF-IDF vectorization
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=1, max_df=0.9, stop_words='english')
X_tfidf = tfidf.fit_transform(df_oversampled['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_oversampled['Sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)



# 2. Train and evaluate Voting Classifier
# Define the models for VotingClassifier
model1 = LogisticRegression(max_iter=1000)
model2 = RandomForestClassifier()
model3 = SVC(probability=True)

# Initialize the Voting Classifier
voting_model = VotingClassifier(estimators=[
    ('lr', model1), ('rf', model2), ('svc', model3)
], voting='soft')

# Train the Voting Classifier
voting_model.fit(X_train, y_train)

# Make predictions with Voting Classifier
voting_predictions = voting_model.predict(X_test)

# Calculate Voting Classifier accuracy
voting_accuracy = accuracy_score(y_test, voting_predictions)
print(f'Voting Classifier Accuracy: {voting_accuracy * 100:.2f}%')

# Optional: You can calculate other metrics like precision, recall, and F1 score for both models
# Logistic Regression metrics
logreg_precision = precision_score(y_test, logreg_predictions, average='weighted')
logreg_recall = recall_score(y_test, logreg_predictions, average='weighted')
logreg_f1 = f1_score(y_test, logreg_predictions, average='weighted')

print(f'Logistic Regression Precision: {logreg_precision * 100:.2f}%')
print(f'Logistic Regression Recall: {logreg_recall * 100:.2f}%')
print(f'Logistic Regression F1 Score: {logreg_f1 * 100:.2f}%')

# Voting Classifier metrics
voting_precision = precision_score(y_test, voting_predictions, average='weighted')
voting_recall = recall_score(y_test, voting_predictions, average='weighted')
voting_f1 = f1_score(y_test, voting_predictions, average='weighted')

print(f'Voting Classifier Precision: {voting_precision * 100:.2f}%')
print(f'Voting Classifier Recall: {voting_recall * 100:.2f}%')
print(f'Voting Classifier F1 Score: {voting_f1 * 100:.2f}%')


In [ ]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score




# Convert text data into features using TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=10000, min_df=2, max_df=0.8, stop_words='english')
X_tfidf = tfidf.fit_transform(filtered_df['cleaned_review'])

# Encode the 'Sentiment' column to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(filtered_df['Sentiment'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Initialize the Naive Bayes classifier
model = MultinomialNB()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


# New Reviews to predict their sentiments
new_reviews = [
    "The doctor was incredibly attentive and took the time to explain everything.",
    "I had a great experience at the hospital; the staff were kind and professional.",
    "The surgery went smoothly, and the recovery process was well-managed.",
    "The nurses were compassionate and made me feel comfortable during my stay.",
    "I was seen quickly, and the treatment was effective. Highly recommend this clinic.",
    # Add more new reviews here...
]

# Ensure the vectorizer is not refitted, only transform the new data
new_tfidf_features = tfidf.transform(new_reviews)

# Predict sentiment using the trained MultinomialNB model
new_predictions = model.predict(new_tfidf_features)

# Decode the numeric predictions back to original labels
decoded_predictions = label_encoder.inverse_transform(new_predictions)

print(decoded_predictions)
